In [9]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tqdm import tqdm
import os
import math
from collections import Counter
from scipy.sparse import csr_matrix
from sys import getsizeof
import numpy as np
from sklearn.preprocessing import normalize

**COMPUTE IDF**

In [10]:
def compidf(uniq,d, data):
    c=0
    lstc=[]
    for u in uniq:    
        for i in d:
            for j in i.keys():
                if u== j:
                    c+=1 #count of a word in the whole corpus
        lstc.append(c) # list of count of a word in the whole corpus
        c=0
    wordcount=dict(zip(uniq,lstc)) # dict of word and their count
    idf_val=[]
    for i, j in wordcount.items():
        idf= 1+ math.log((1+len(data))/(1+j)) #formula for IDF
        idf_val.append(idf)
    idfdict= dict(zip(uniq,idf_val))
    return idf_val, idfdict

In [11]:
def fit(data):
    d=[]
    unique_words=set()
    if isinstance(data,list):
        for i in data:
            d.append(dict(Counter(i.split()))) # d has dicts with words in the data as keys and freq as their values
            for j in i.split(" "):
                if len(j)<2:
                    continue
                unique_words.add(j)
        unique_words= sorted(list(unique_words)) # it is a set of all the unique words
        vocab= {j:i for i,j in enumerate(unique_words)} # it is a dict with keys as words and values as indexes
        idf_val, idfdict= compidf(unique_words,d, data)
        return unique_words, idf_val, idfdict, vocab,d
    else:
        print("wrong datatype")
    

In [12]:
data = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [13]:
uniq, idf_val, idfdict, vocab,d= fit(data)

In [7]:
d

[{'this': 1, 'is': 1, 'the': 1, 'first': 1, 'document': 1},
 {'this': 1, 'document': 2, 'is': 1, 'the': 1, 'second': 1},
 {'and': 1, 'this': 1, 'is': 1, 'the': 1, 'third': 1, 'one': 1},
 {'is': 1, 'this': 1, 'the': 1, 'first': 1, 'document': 1}]

In [6]:
idfdict["this"]

1.0

**Computing TF**

In [14]:
def comptf(d):
    l=[]
    listi=[]
    alpha=[]
    alphai=[]
    for k in range(len(d)):    
        for i,j in d[k].items():
            tf= j/sum(d[k].values()) #formula for term frequency
            l.append(tf) #appending tf values
            listi.append(i) # appending words
        alpha.append(l) #appending list of tf values
        alphai.append(listi) #appending list of words
        l=[]
        listi=[]
    return alphai, alpha

In [13]:
a,b=comptf(d)

In [14]:
a

[['this', 'is', 'the', 'first', 'document'],
 ['this', 'document', 'is', 'the', 'second'],
 ['and', 'this', 'is', 'the', 'third', 'one'],
 ['is', 'this', 'the', 'first', 'document']]

In [8]:
b

[[0.2, 0.2, 0.2, 0.2, 0.2],
 [0.16666666666666666,
  0.3333333333333333,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666],
 [0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666],
 [0.2, 0.2, 0.2, 0.2, 0.2]]

**COMPUTE IDF**

In [18]:
def compidf(uniq,d):
    c=0
    lstc=[]
    for u in uniq:    
        for i in d:
            for j in i.keys():
                if u== j:
                    c+=1 #count of a word in the whole corpus
        lstc.append(c) # list of count of a word in the whole corpus
        c=0
    wordcount=dict(zip(uniq,lstc)) # dict of word and their count
    final=[]
    for i, j in wordcount.items():
        idf= 1+ math.log((1+len(data))/(1+j)) #formula for IDF
        final.append(idf)
    idfdict= dict(zip(uniq,final)) #zipping word with its IDF value
    return idfdict

In [285]:
final= compidf(uniq,d)

**COMPUTING A LIST OF DICTIONARIES WITH WORDS AS KEYS AND THEIR TF VALUES AS VALUES**

In [40]:
tflist=d.copy()
for idx, val in enumerate(b):
    #print(idx, val)
    zipper= list(zip(a[idx], val))
    for i in zipper:
        tflist[idx][i[0]]=i[1]
print(tflist)

[{'this': 0.2, 'is': 0.2, 'the': 0.2, 'first': 0.2, 'document': 0.2}, {'this': 0.16666666666666666, 'document': 0.3333333333333333, 'is': 0.16666666666666666, 'the': 0.16666666666666666, 'second': 0.16666666666666666}, {'and': 0.16666666666666666, 'this': 0.16666666666666666, 'is': 0.16666666666666666, 'the': 0.16666666666666666, 'third': 0.16666666666666666, 'one': 0.16666666666666666}, {'is': 0.2, 'this': 0.2, 'the': 0.2, 'first': 0.2, 'document': 0.2}]


In [10]:
d

[{'this': 0.2, 'is': 0.2, 'the': 0.2, 'first': 0.2, 'document': 0.2},
 {'this': 0.16666666666666666,
  'document': 0.3333333333333333,
  'is': 0.16666666666666666,
  'the': 0.16666666666666666,
  'second': 0.16666666666666666},
 {'and': 0.16666666666666666,
  'this': 0.16666666666666666,
  'is': 0.16666666666666666,
  'the': 0.16666666666666666,
  'third': 0.16666666666666666,
  'one': 0.16666666666666666},
 {'is': 0.2, 'this': 0.2, 'the': 0.2, 'first': 0.2, 'document': 0.2}]

In [76]:
for i,j in enumerate(tflist):
    print(j["this"])
    print()
    for word, tfval in j.items():
        print(j["this"])
        print()
        #print(word, tfval)

0.2

0.2

0.2

0.2

0.2

0.2

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.16666666666666669

0.2

0.2

0.2

0.2

0.2

0.2



**MULTIPLICATION OF TF AND IDF**

In [366]:
tfidflist= tflist.copy()
op=[]
for idx, val in enumerate(tfidflist):
    for key, kval in val.items():
        nval= final[key]* kval
        op.append(nval)
print(op)

[0.2, 0.2, 0.2, 0.3021651247531982, 0.24462871026284194, 0.16666666666666666, 0.40771451710473655, 0.16666666666666666, 0.16666666666666666, 0.3193817886456925, 0.3193817886456925, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.3193817886456925, 0.3193817886456925, 0.2, 0.2, 0.2, 0.3021651247531982, 0.24462871026284194]


In [15]:
def transform(dataset,vocab,d):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        a,b=comptf(d)
        tflist=d.copy()
        for idx, val in enumerate(b):
            zipper= list(zip(a[idx], val))
            for i in zipper:
                tflist[idx][i[0]]=i[1]
        for idx,row in enumerate(tflist):# for each dict in the tflist
            for word, tfval in row.items():
            # it will return a dict type object where key is the word and values is its TF Value, {word:TF Value}                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the Mult of TF and IDF of the word
                    tfidf= tfval*idfdict[word]
                    values.append(tfidf)
        return normalize(csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab))))
    else:
        print("you need to pass list of strings")

In [16]:
yup= transform(data,vocab,d)

In [17]:
print(yup[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [18]:
yup.shape

(4, 9)